In [19]:
# This section can be uncommented if running on a system without required modules (e.g., UA HPC)
'''import sys
!{sys.executable} -m pip install rasterio'''

import rasterio
from rasterio.plot import show
from rasterio.plot import show_hist
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import csv
from datetime import datetime

In [20]:
# Read in the DSM
img_ref = "7_18_2024_Gantry_North_P4_15m_RGB_dsm.tif"
src_ref = rasterio.open(img_ref)
arr_ref = src_ref.read(1)

# Define output filenames
img_output = img_ref + "_adjusted.tif"
log_output = img_ref + "_adjusted_log.txt"

with open(log_output, 'w') as output_file:
    output_file.write("img_ref: " + img_ref + "\n")
    output_file.write("img_output: " + img_output + "\n")
    output_file.write("log_output: " + log_output + "\n")

In [21]:
# Here you have two options to provide reference positions
# You can either pass in a CSV or manually define the positions.
# Whatever positions are used must be completely unobstructed at the center of the GCP to avoid skewing the results.
# To be sure, check the RGB orthomosaic and ensure the GCP isn't obstructed.


'''# Read in the CSV of GCP locations
# Replace 'your_file.csv' with the actual filename
csv_filename = 'your_file.csv'

# Initialize empty lists for each column
x = []
y = []
z = []

with open(csv_filename, 'r') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        # Assuming the second column is at index 1 (Python uses 0-based indexing)
        x.append(float(row[1]))
        # Third column (index 2)
        y.append(float(row[2]))
        # Fourth column (index 3)
        z.append(float(row[3]))
'''        

# Manually define positions
x = [408991.775, 408993.42, 409015.356]
y = [3660202.957, 3660363.1, 3660202.363]
z = [360.316, 360.348, 360.377]

'''x = [408991.775, 408993.42, 409016.773, 409015.356]
y = [3660202.957, 3660363.1, 3660362.973, 3660202.363]
z = [360.316, 360.348, 360.324, 360.377]'''

# GCPs 1-4 good through 2024-06-06; GCP 3 obstructed afterwards 


with open(log_output, 'a') as output_file:
    output_file.write("x: " + "'" + "','".join(map(str, x)) + "'" + "\n")
    output_file.write("y: " + "'" + "','".join(map(str, y)) + "'" + "\n")
    output_file.write("z: " + "'" + "','".join(map(str, z)) + "'" + "\n")

In [22]:
diff = []

# Get the corresponding pixels at each location
for i in range(len(x)):
    py, px = src_ref.index(x[i], y[i])
    window = rasterio.windows.Window(px - 1//2, py - 1//2, 1, 1)
    clip = src_ref.read(window=window)
    print("The value read at this point is", round(clip[0][0][0],3))
    print("The expected value is", z[i])
    print("The difference is", round(clip[0][0][0] - z[i],3))
    diff.append(round(clip[0][0][0] - z[i],3))
    
mean_diff = np.mean(diff)
if mean_diff < 0:
    print("On average, the value in the DSM is lower than expected by ", abs(mean_diff))
else:
    print("On average, the value in the DSM is higher than expected by ", abs(mean_diff))
    
with open(log_output, 'a') as output_file:
    # Write the result to the file
    output_file.write("diffs: " + "'" + "','".join(map(str, diff)) + "'" + "\n")
    output_file.write("mean_diff: " + str(mean_diff) + "\n")

The value read at this point is 360.369
The expected value is 360.316
The difference is 0.053
The value read at this point is 360.309
The expected value is 360.348
The difference is -0.039
The value read at this point is 360.393
The expected value is 360.377
The difference is 0.016
On average, the value in the DSM is higher than expected by  0.01


In [23]:
# We want to shift all points that are not -10000 down by this value.
print("The max of the input is", arr_ref.max())
print("The min of the input is", arr_ref.min())

arr_ref_out = arr_ref # set values the same initially
arr_ref_out = arr_ref - mean_diff

# Fix the nodata cells
arr_ref_out[arr_ref_out<-1000] = -10000

print("The max of the output is", arr_ref_out.max())
print("The min of the output is", arr_ref_out.min())


with open(log_output, 'a') as output_file:
    output_file.write("input max: " + str(arr_ref.max()) + "\n")
    output_file.write("input min: " + str(arr_ref.min()) + "\n")
    output_file.write("output max: " + str(arr_ref_out.max()) + "\n")
    output_file.write("output min: " + str(arr_ref_out.min()) + "\n")

The max of the input is 364.2312
The min of the input is -10000.0
The max of the output is 364.2212
The min of the output is -10000.0


In [24]:
with rasterio.open(img_output, 'w', **src_ref.profile) as dst:
    dst.write(arr_ref_out, indexes = 1)
    
with open(log_output, 'a') as output_file:
    output_file.write("completed: " + str(datetime.now()) + "\n")